In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 7, 7), datetime.date(2023, 7, 6))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-07-07
Today in mm-dd format: 07-07
yesterday: 2023-07-06 00:00:00


In [3]:
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-06-08 00:00:00   Thursday


In [4]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [5]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART'"

In [6]:
in_p = in_p + ", 'BEM', 'EA'"
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA'"

### Bypass this statement if aboce statement is executed

In [8]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'GULF', 'TTB'"

### End of Selection between Buy or Orders

In [7]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-06-08 00:00:00' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA') 
ORDER BY name, date


In [8]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
322,JMART,2023-06-28,13.60,31204314,13.90,13.20
684,WHART,2023-06-12,11.20,1052392,11.20,11.00
247,ICHI,2023-06-15,15.00,4615457,15.40,15.00
217,EA,2023-07-05,54.25,11778894,56.00,54.00
353,KCE,2023-06-09,39.25,4813415,39.75,39.00


In [9]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-06-08|2023-06-08|1|36.5|36.5|0.0|0.0|0|0|37.75|27.75||2|88,779,944
AH|2023-06-09|2023-06-09|1|34.5|34.5|0.0|0.0|0|0|37.25|34.25|Downward|3|15,050,320
AH|2023-06-12|2023-06-12|1|35.25|35.25|0.0|0.0|0|0|36.25|34.75|Upward|4|7,101,473
AH|2023-06-13|2023-06-14|2|35.0|34.5|-0.5|-1.43|-2|-1|35.75|33.75|Downward|6|5,935,722
AH|2023-06-15|2023-06-15|1|34.75|34.75|0.0|0.0|0|0|35.25|34.5|Upward|7|1,501,200
AH|2023-06-16|2023-06-21|4|34.5|32.75|-1.75|-5.07|-7|-2|35.0|32.25|Downward|11|8,367,251
AH|2023-06-22|2023-06-22|1|33.25|33.25|0.0|0.0|0|0|33.75|32.25|Upward|12|2,011,971
AH|2023-06-23|2023-06-26|2|32.25|30.5|-1.75|-5.43|-7|-4|33.0|30.5|Downward|14|6,611,429
AH|2023-06-27|2023-07-03|5|32.25|35.0|2.75|8.53|11|2|35.0|30.75|Upward|19|10,657,316
AH|2023-07-04|2023-07-04|1|34.5|34.5|0.0|0.0|0|0|35.0|34.25|Downward|20|689,571
AH|2023-07-05|2023-07-07|3|

KCE|2023-07-03|2023-07-07|5|38.5|36.5|-2.0|-5.19|-8|-2|39.75|36.25|Downward|375|31,077,749
KTC|2023-06-08|2023-06-08|1|52.75|52.75|0.0|0.0|0|0|52.75|50.75|Downward|376|7,220,533
KTC|2023-06-09|2023-06-09|1|53.0|53.0|0.0|0.0|0|0|53.0|52.25|Upward|377|2,410,836
KTC|2023-06-12|2023-06-12|1|51.5|51.5|0.0|0.0|0|0|52.75|51.5|Downward|378|3,219,169
KTC|2023-06-13|2023-06-13|1|52.25|52.25|0.0|0.0|0|0|52.5|51.5|Upward|379|2,038,613
KTC|2023-06-14|2023-06-15|2|51.5|49.5|-2.0|-3.88|-8|-4|52.5|49.5|Downward|381|13,335,769
KTC|2023-06-16|2023-06-16|1|50.25|50.25|0.0|0.0|0|0|50.5|49.25|Upward|382|5,252,490
KTC|2023-06-19|2023-06-19|1|49.5|49.5|0.0|0.0|0|0|50.0|49.25|Downward|383|2,523,417
KTC|2023-06-20|2023-06-23|4|50.25|50.5|0.25|0.5|1|0|51.25|49.5|Upward|387|15,842,414
KTC|2023-06-26|2023-06-28|3|49.75|47.75|-2.0|-4.02|-8|-3|51.0|47.5|Downward|390|16,039,659
KTC|2023-06-29|2023-07-04|4|48.25|50.25|2.0|4.15|8|2|50.25|47.25|Upward|394|20,437,017
KTC|2023-07-05|2023-07-07|3|49.25|46.0|-3.25|-6.6|-13

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
74,DIF,2023-06-08,2023-06-08,1,12.40,12.40,0.00,0.00%,0,0,12.40,12.30,"15,489,235",Upward,178
225,SINGER,2023-06-29,2023-06-29,1,9.05,9.05,0.00,0.00%,0,0,9.20,8.35,"16,942,470",Upward,567
222,SINGER,2023-06-23,2023-06-26,2,9.85,8.25,-1.60,-16.24%,-32,-16,9.85,8.20,"25,093,285",Downward,564
132,JMT,2023-06-21,2023-06-26,4,38.75,33.50,-5.25,-13.55%,-21,-6,41.00,33.50,"78,721,052",Downward,344
254,TFFIF,2023-06-20,2023-06-21,2,7.75,7.75,0.00,0.00%,0,0,7.80,7.70,"3,026,647",Upward,627


In [15]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [16]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
10,AH,07-05,07-07,3,35.00,35.75,2.14%,3,1,35.75,34.25,"4,153,754",Upward
73,CPNREIT,07-05,07-07,3,10.80,11.00,1.85%,2,0,11.00,10.70,"7,946,165",Upward
120,JASIF,07-07,07-07,1,7.05,7.05,0.00%,0,0,7.10,7.05,"637,260",Downward
270,TMT,07-05,07-07,3,7.40,7.40,0.00%,0,0,7.50,7.35,"153,492",Downward
260,TFFIF,07-03,07-07,5,7.60,7.60,0.00%,0,0,7.65,7.55,"9,231,127",Upward


In [17]:
name_up = 'AH'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
0,AH,06-08,06-08,1,36.50,36.50,0.00%,0,0,37.75,27.75,"88,779,944",nan
1,AH,06-09,06-09,1,34.50,34.50,0.00%,0,0,37.25,34.25,"15,050,320",Downward
2,AH,06-12,06-12,1,35.25,35.25,0.00%,0,0,36.25,34.75,"7,101,473",Upward
3,AH,06-13,06-14,2,35.00,34.50,-1.43%,-2,-1,35.75,33.75,"5,935,722",Downward
4,AH,06-15,06-15,1,34.75,34.75,0.00%,0,0,35.25,34.50,"1,501,200",Upward
5,AH,06-16,06-21,4,34.50,32.75,-5.07%,-7,-2,35.00,32.25,"8,367,251",Downward
6,AH,06-22,06-22,1,33.25,33.25,0.00%,0,0,33.75,32.25,"2,011,971",Upward
7,AH,06-23,06-26,2,32.25,30.50,-5.43%,-7,-4,33.00,30.50,"6,611,429",Downward
8,AH,06-27,07-03,5,32.25,35.00,8.53%,11,2,35.00,30.75,"10,657,316",Upward
9,AH,07-04,07-04,1,34.50,34.50,0.00%,0,0,35.00,34.25,"689,571",Downward


In [18]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
158,KTC,07-05,07-07,3,49.25,46.00,-6.60%,-13,-5,50.00,45.50,"23,681,010",Downward
147,KCE,07-03,07-07,5,38.50,36.50,-5.19%,-8,-2,39.75,36.25,"31,077,749",Downward
40,AWC,07-05,07-07,3,4.82,4.60,-4.56%,-11,-4,4.94,4.54,"133,658,043",Downward
88,EA,07-03,07-07,5,56.25,54.00,-4.00%,-9,-2,57.25,53.00,"41,914,683",Downward
20,ASK,07-05,07-07,3,25.75,25.00,-2.91%,-3,-1,26.00,24.70,"1,077,038",Downward


In [19]:
name_down = 'KTC'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
148,KTC,06-08,06-08,1,52.75,52.75,0.00%,0,0,52.75,50.75,"7,220,533",Downward
149,KTC,06-09,06-09,1,53.00,53.00,0.00%,0,0,53.00,52.25,"2,410,836",Upward
150,KTC,06-12,06-12,1,51.50,51.50,0.00%,0,0,52.75,51.50,"3,219,169",Downward
151,KTC,06-13,06-13,1,52.25,52.25,0.00%,0,0,52.50,51.50,"2,038,613",Upward
152,KTC,06-14,06-15,2,51.50,49.50,-3.88%,-8,-4,52.50,49.50,"13,335,769",Downward
153,KTC,06-16,06-16,1,50.25,50.25,0.00%,0,0,50.50,49.25,"5,252,490",Upward
154,KTC,06-19,06-19,1,49.50,49.50,0.00%,0,0,50.00,49.25,"2,523,417",Downward
155,KTC,06-20,06-23,4,50.25,50.50,0.50%,1,0,51.25,49.50,"15,842,414",Upward
156,KTC,06-26,06-28,3,49.75,47.75,-4.02%,-8,-3,51.00,47.50,"16,039,659",Downward
157,KTC,06-29,07-04,4,48.25,50.25,4.15%,8,2,50.25,47.25,"20,437,017",Upward


In [20]:
df_groupby_trend = df.groupby('trend')

In [21]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
87,EA,06-20,06-20,1,61.75,61.75,0.00%,0,0,62.75,61.75,"7,163,732",Upward
112,IVL,06-08,06-14,5,33.25,35.00,5.26%,7,1,35.50,32.00,"96,558,242",Upward
175,NER,06-22,06-22,1,4.70,4.70,0.00%,0,0,4.76,4.68,"3,244,398",Upward
208,RCL,06-29,07-04,4,23.70,24.50,3.38%,8,2,24.60,23.20,"8,464,802",Upward
246,STA,06-29,06-30,2,17.00,17.00,0.00%,0,0,17.60,16.70,"11,230,604",Upward


In [22]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
92,EA,06-28,06-28,1,52.25,52.25,0.00%,0,0,53.75,51.50,"17,527,870",Downward
222,SCC,07-06,07-06,1,323.00,323.00,0.00%,0,0,326.00,320.00,"2,143,777",Downward
218,SCC,06-23,06-27,3,325.00,321.00,-1.23%,-4,-2,328.00,321.00,"3,123,060",Downward
282,TMT,07-05,07-06,2,7.40,7.40,0.00%,0,0,7.50,7.35,"147,090",Downward
108,ICHI,06-15,06-27,9,15.00,12.90,-14.00%,-21,-3,15.40,12.70,"92,708,282",Downward


In [23]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,12,161,2.00,2.52
Upward,6,141,2.00,2.11


In [24]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [25]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
128,JMART,06-07,06-14,6,19.20,20.10,4.69%,9,1,20.50,18.90,"97,232,869",Upward


In [26]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
79,DIF,06-09,06-26,12,12.30,10.60,-13.82,-17,-2,12.40,10.00,439470965,Downward
167,MCS,06-14,06-29,12,7.10,6.15,-13.38,-19,-2,7.15,5.95,7826524,Downward


### Individual stock analysis

In [27]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
0,AH,06-07,06-07,1,39.25,39.25,0.00%,0,0,39.25,37.50,"4,594,579",nan
1,AH,06-08,06-09,2,36.50,34.50,-5.48%,-8,-4,37.75,27.75,"103,830,264",Downward
2,AH,06-12,06-12,1,35.25,35.25,0.00%,0,0,36.25,34.75,"7,101,473",Upward
3,AH,06-13,06-14,2,35.00,34.50,-1.43%,-2,-1,35.75,33.75,"5,935,722",Downward
4,AH,06-15,06-15,1,34.75,34.75,0.00%,0,0,35.25,34.50,"1,501,200",Upward
5,AH,06-16,06-21,4,34.50,32.75,-5.07%,-7,-2,35.00,32.25,"8,367,251",Downward
6,AH,06-22,06-22,1,33.25,33.25,0.00%,0,0,33.75,32.25,"2,011,971",Upward
7,AH,06-23,06-26,2,32.25,30.50,-5.43%,-7,-4,33.00,30.50,"6,611,429",Downward
8,AH,06-27,07-03,5,32.25,35.00,8.53%,11,2,35.00,30.75,"10,657,316",Upward
9,AH,07-04,07-04,1,34.50,34.50,0.00%,0,0,35.00,34.25,"689,571",Downward


In [28]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,22.00,22.00,22.00,22.00
mean,34.18,6988777.14,34.86,33.17
std,1.77,18515301.65,1.71,1.94
min,30.50,689571.00,32.50,27.75
25%,33.06,1833894.75,33.81,32.25
50%,34.50,2116167.50,35.00,33.50
75%,35.00,3191044.00,35.25,34.25
max,39.25,88779944.00,39.25,37.50


In [29]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
154,KTC,06-07,06-07,1,51.00,51.00,0.00%,0,0,51.00,50.00,"5,391,248",Downward
155,KTC,06-08,06-09,2,52.75,53.00,0.47%,1,0,53.00,50.75,"9,631,369",Upward
156,KTC,06-12,06-12,1,51.50,51.50,0.00%,0,0,52.75,51.50,"3,219,169",Downward
157,KTC,06-13,06-13,1,52.25,52.25,0.00%,0,0,52.50,51.50,"2,038,613",Upward
158,KTC,06-14,06-15,2,51.50,49.50,-3.88%,-8,-4,52.50,49.50,"13,335,769",Downward
159,KTC,06-16,06-16,1,50.25,50.25,0.00%,0,0,50.50,49.25,"5,252,490",Upward
160,KTC,06-19,06-19,1,49.50,49.50,0.00%,0,0,50.00,49.25,"2,523,417",Downward
161,KTC,06-20,06-23,4,50.25,50.50,0.50%,1,0,51.25,49.50,"15,842,414",Upward
162,KTC,06-26,06-28,3,49.75,47.75,-4.02%,-8,-3,51.00,47.50,"16,039,659",Downward
163,KTC,06-29,07-04,4,48.25,50.25,4.15%,8,2,50.25,47.25,"20,437,017",Upward


In [30]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,22.00,22.00,22.00,22.00
mean,50.12,4957339.86,50.88,49.56
std,1.56,3034960.29,1.19,1.45
min,46.25,1556937.00,49.00,46.00
25%,49.50,2605629.50,50.06,49.06
50%,50.25,4495947.00,50.50,49.50
75%,50.88,5598271.00,51.62,50.19
max,53.00,13793375.00,53.00,52.25


In [31]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [32]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [33]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-07-06' 
ORDER BY name, date


In [34]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-07-06,1.89,16612072,1.91,1.88
1,ADVANC,2023-07-06,216.00,4809055,219.00,215.00
2,AEONTS,2023-07-06,170.00,1212666,179.00,167.50
3,AH,2023-07-06,35.00,1311614,35.25,34.50
4,AIMIRT,2023-07-06,11.50,118234,11.50,11.40
...,...,...,...,...,...,...
182,WHAIR,2023-07-06,7.05,637600,7.10,7.00
183,WHART,2023-07-06,10.50,553189,10.60,10.50
184,WHAUP,2023-07-06,3.58,504879,3.60,3.56
185,WICE,2023-07-06,8.65,803767,8.70,8.40
